<a href="https://colab.research.google.com/github/AamirKhaan/Student-Academic-Performance/blob/master/01_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Image Manipulation

Manipulation of raw mobile phone captured images.

In [1]:
import glob
import os

import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from tensorflow.keras.utils import get_file
from PIL import Image
import time
from IPython.display import clear_output

In [2]:
CUSTOM_MODEL_NAME = 'alpr_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('codes', 'workspace'),
    'SCRIPTS_PATH': os.path.join('codes','scripts'),
    'APIMODEL_PATH': os.path.join('codes','models'),
    'ANNOTATION_PATH': os.path.join('codes', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('codes', 'workspace','images'),
    'MODEL_PATH': os.path.join('codes', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('codes', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('codes', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('codes', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('codes', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('codes', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('codes','protoc')
 }

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('codes', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
#!pip uninstall tensorflow-object-detection-api -y

In [ ]:
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin')) 
!cd codes/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
!cd codes/models/research/slim && pip install -e .

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
!pip uninstall pyparsing -y

In [ ]:
!pip uninstall object_detection.builders -y

In [ ]:
!pip install object_detection.builders

In [ ]:
!pip install pyparsing==2.4.2

# 8. Load Train Model From Checkpoint

In [ ]:
!pip install tensorflow --upgrade

In [ ]:
#!pip uninstall protobuf matplotlib -y
#!pip install protobuf matplotlib==3.2

In [ ]:
#!pip install tensorflow-object-detection-api

In [5]:
import os
import tensorflow as tf
import object_detection
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [6]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-11')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 9. Detect from an Image

In [7]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [9]:
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', '*.jpg')

In [10]:
images_path= list(glob.glob(IMAGE_PATH))

In [11]:
print(f'There are {len(images_path):,} images in the dataset')

There are 64 images in the dataset


In [12]:
for image in images_path:

    img = cv2.imread(image)
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)



    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    # print(detections['detection_boxes'])
    # print(detections['detection_classes'])
    # print(detections['detection_scores'])

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.2,
                agnostic_mode=False)
    
    img = Image.fromarray(image_np_with_detections[...,::-1])
    display(img)
#     print(type(image_np_with_detections))
#     plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
#     plt.show()
    time.sleep(5)
    clear_output(wait=False)

### 1. Raw Image path.

In [ ]:
IMAGE_PATH ='data/resized/*.jpg'

In [ ]:
images_path= list(glob.glob(IMAGE_PATH))

In [ ]:
print(f'There are {len(image_paths):,} images in the dataset')

In [ ]:
for image in image_paths:
    img=Image.open(image)
    img = img.rotate(-90)

    width, height = img.size
    img = img.resize((width //5 ,height // 5), resample = Image.BICUBIC)

    display(img)
    print(img.size)
    time.sleep(5)
    clear_output(wait=False)

In [ ]:
output_image_path = 'data/resized/car_'

In [ ]:
for i,image in enumerate(image_paths):
    img=Image.open(image)
    img = img.rotate(-90)

    width, height = img.size
    img = img.resize((width //5 ,height // 5), resample = Image.BICUBIC)

    #display(img)
    //print(img.size)
    img.save(output_image_path+str(i)+'.jpg')
    #time.sleep(5)
    #clear_output(wait=False)

In [ ]:
Image.open(image_paths[0])
img = image.rotate(-90)

width, height = img.size
img = img.resize((width //5 ,height // 5), resample = Image.BICUBIC)

display(img)
print(img.size)
time.sleep(2)